In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler

# Leitura e Tratamento dos Dados

In [ ]:
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')
print(f'Train set shape: {train.shape}\nTest set shape: {test.shape}')

In [ ]:
train.head()

In [ ]:
train['Embarked'].value_counts()

In [ ]:
train['Sex'].value_counts()

In [ ]:
train['Sex_b'] = train['Sex'].map(lambda x: 1 if x == 'female' else 0)
test['Sex_b'] = test['Sex'].map(lambda x: 1 if x == 'female' else 0)

In [ ]:
train['Sex_b'].value_counts()

In [ ]:
list(train['Cabin'].unique())

In [ ]:
train['Embarked_S'] = (train['Embarked'] == 'S').astype(int)
train['Embarked_C'] = (train['Embarked'] == 'C').astype(int)

train['Cabin_null'] = train['Cabin'].isnull().astype(int)
train['Cabin_C'] = train['Cabin'].fillna('').str.contains('C').astype(int)
train['Cabin_E'] = train['Cabin'].fillna('').str.contains('E').astype(int)
train['Cabin_G'] = train['Cabin'].fillna('').str.contains('G').astype(int)
train['Cabin_D'] = train['Cabin'].fillna('').str.contains('D').astype(int)
train['Cabin_A'] = train['Cabin'].fillna('').str.contains('A').astype(int)
train['Cabin_B'] = train['Cabin'].fillna('').str.contains('B').astype(int)
train['Cabin_F'] = train['Cabin'].fillna('').str.contains('F').astype(int)
train['Cabin_T'] = train['Cabin'].fillna('').str.contains('T').astype(int)

train['Name_Miss'] = train['Name'].str.contains('Miss.').astype(int)
train['Name_Mrs'] = train['Name'].str.contains('Mrs.').astype(int)
train['Name_Master'] = train['Name'].str.contains('Master.').astype(int)
train['Name_Col'] = train['Name'].str.contains('Col.').astype(int)
train['Name_Major'] = train['Name'].str.contains('Major.').astype(int)
train['Name_Mr'] = train['Name'].str.contains('Mr.').astype(int)
train['Name_Dr'] = train['Name'].str.contains('Dr.').astype(int)
train['Name_Don'] = train['Name'].str.contains('Don.').astype(int)
train['Name_Sir'] = train['Name'].str.contains('Sir.').astype(int)

In [ ]:
test['Embarked_S'] = (test['Embarked'] == 'S').astype(int)
test['Embarked_C'] = (test['Embarked'] == 'C').astype(int)

#test['Cabin_null'] = test['Cabin'].map(lambda x: 0 if x is np.nan else len(str(x)))
test['Cabin_null'] = test['Cabin'].isnull().astype(int)
test['Cabin_C'] = test['Cabin'].fillna('').str.contains('C').astype(int)
test['Cabin_E'] = test['Cabin'].fillna('').str.contains('E').astype(int)
test['Cabin_G'] = test['Cabin'].fillna('').str.contains('G').astype(int)
test['Cabin_D'] = test['Cabin'].fillna('').str.contains('D').astype(int)
test['Cabin_A'] = test['Cabin'].fillna('').str.contains('A').astype(int)
test['Cabin_B'] = test['Cabin'].fillna('').str.contains('B').astype(int)
test['Cabin_F'] = test['Cabin'].fillna('').str.contains('F').astype(int)
test['Cabin_T'] = test['Cabin'].fillna('').str.contains('T').astype(int)

test['Name_Miss'] = test['Name'].str.contains('Miss.').astype(int)
test['Name_Mrs'] = test['Name'].str.contains('Mrs.').astype(int)
test['Name_Master'] = test['Name'].str.contains('Master.').astype(int)
test['Name_Col'] = test['Name'].str.contains('Col.').astype(int)
test['Name_Major'] = test['Name'].str.contains('Major.').astype(int)
test['Name_Mr'] = test['Name'].str.contains('Mr.').astype(int)
test['Name_Dr'] = test['Name'].str.contains('Dr.').astype(int)
test['Name_Don'] = test['Name'].str.contains('Don.').astype(int)
test['Name_Sir'] = test['Name'].str.contains('Sir.').astype(int)

In [ ]:
variaveis = ['Sex_b', 'Age', 'Pclass', 'Embarked_S', 'Embarked_C', 'SibSp', 'Parch', 'Fare', 'Cabin_null',
             'Cabin_C', 'Cabin_E', 'Cabin_G', 'Cabin_D', 'Cabin_A', 'Cabin_B', 'Cabin_F', 'Cabin_T',
             'Name_Miss', 'Name_Mrs', 'Name_Master', 'Name_Col', 'Name_Major', 'Name_Mr', 'Name_Dr', 'Name_Don', 'Name_Sir']

In [ ]:
X = train[variaveis].fillna(-1)
X_test = test[variaveis].fillna(-1)
y = train['Survived']

# Regressão Logística

In [ ]:
kf = RepeatedKFold(n_splits=3, n_repeats=10, random_state=42)

scaler = StandardScaler()

step_train = []
step = []

for linhas_train, linhas_cv in kf.split(X):
    X_train, X_cv = X.iloc[linhas_train].copy(), X.iloc[linhas_cv].copy()
    y_train, y_cv = y.iloc[linhas_train].copy(), y.iloc[linhas_cv].copy()

    X_train = scaler.fit_transform(X_train)
    X_cv = scaler.transform(X_cv)

    model = LogisticRegression(penalty='l2', random_state=42)
    model.fit(X_train, y_train)

    yhat_train = model.predict(X_train)
    yhat = model.predict(X_cv)

    acc_train = np.mean(y_train == yhat_train)
    acc = np.mean(y_cv == yhat)
    print(f'acc_train: {acc_train}, acc_cv: {acc}\n')

    step_train.append(acc_train)
    step.append(acc)

In [ ]:
print(f'Train mean: {np.mean(step_train)}, CV mean: {np.mean(step)}')

In [ ]:
pylab.hist(step), pylab.hist(step_train, alpha=.8)

## Análise de Erro

In [ ]:
X_cv_erro = train.iloc[linhas_cv].copy()
X_cv_erro['yhat'] = yhat
X_cv_erro.head()

In [ ]:
erro = X_cv_erro[X_cv_erro['Survived'] != X_cv_erro['yhat']]
erro = erro[['PassengerId', 'Survived', 'yhat', 'Name', 'Sex', 'Sex_b', 'Embarked', 'Age', 'Ticket', 'Cabin', 'Cabin_null', 'Pclass', 'Embarked_S', 'Embarked_C', 'SibSp', 'Parch', 'Fare', 'Cabin_null',
             'Name_Miss', 'Name_Mrs', 'Name_Master', 'Name_Col', 'Name_Major', 'Name_Mr', 'Name_Dr', 'Name_Don', 'Name_Sir']]
erro.head()

In [ ]:
female = erro[erro['Sex_b'] == 1]
male = erro[erro['Sex_b'] == 0]

In [ ]:
female.sort_values('Survived')

In [ ]:
male.sort_values('Survived')

# Retreinando o Modelo

In [ ]:
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [ ]:
model = LogisticRegression(penalty='l2', random_state=42)
model.fit(X, y)

y_hat = model.predict(X_test)

# Criando o Resultado das Previsões para Importar ao Kaggle

In [ ]:
result = pd.Series(y_hat, index=test['PassengerId'], name='Survived')
result

In [ ]:
result_t.to_csv('./yhat/logistic_regression.csv', header=True)